In [168]:
import string
import re
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 1000)

import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

def warn(*args, **kwargs): pass
import warnings
warnings.warn = warn

In [169]:
data=pd.read_csv('data_rt.csv',sep='|')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102610 entries, 0 to 102609
Data columns (total 2 columns):
label    102610 non-null int64
text     102610 non-null object
dtypes: int64(1), object(1)
memory usage: 1.6+ MB


In [170]:
data.head()

,label,text
0,1,"To an entire generation of filmgoers, it just might represent the most significant leap in storytelling that they will ever see..."
1,1,Pixar classic is one of the best kids' movies of all time.
2,1,"Apesar de representar um imenso avanço tecnológico, a força do filme reside no carisma de seus personagens e no charme de sua história."
3,1,"When Woody perks up in the opening scene, it's not only the toy cowboy who comes alive - we're watching the rebirth of an art form."
4,1,Introduced not one but two indelible characters to the pop culture pantheon: cowboy rag-doll Woody (Tom Hanks) and plastic space ranger Buzz Lightyear (Tim Allen). [Blu-ray]


In [171]:
redundant_signs = set(string.punctuation) - set(['.'])

def clean_data(string):

    string = string.lower()
    
    # fix haven't|doesn't|shouldn't cases
    string = string.replace("n't", " not")
    string = string.replace("'re'", " are")
    
    # here may be actor's names, types of smth etc. I guess it's redundant info
    # let's discuss of necessity of this block
    bracket_words = re.findall('([\(\[\{].+?[\)\]\}])', string)
    for word in bracket_words:
        string = string.replace(''.join(word), "")
        
    # replace redundant_signs
    for item in redundant_signs:
        string = string.replace(item, ' ')
        
    # replace digits
    string = re.sub('\d', ' ', string)
    # replace two or more dots. 1 dot is remained as it separates sentences
    string = re.sub('\.{1,10}', ' ', string)
    # replace one-letter words or just letters
    string = re.sub(r"\b[a-z]{1}\b", ' ', string)
    
    string = ' '.join(list(filter(None, string.split(' '))))
    
    return string

In [172]:
stop_words = list(set(stopwords.words('english')))
stemmer = PorterStemmer()

In [173]:
data['clean_text'] = data['text'].apply(clean_data)

In [174]:
data['clean_text'] = data['clean_text'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop_words]))

In [175]:
letters = [x for x in string.ascii_lowercase + '. ']
data['is_ascii'] = data['clean_text'].apply(lambda x: set(x).issubset(letters))
data['letters'] = data['clean_text'].apply(len)
data['words'] = data['clean_text'].apply(lambda x: len(x.split()))
data['stemed_text'] = data['clean_text'].apply(lambda x: " ".join([stemmer.stem(w) for w in x.split()]))
data['new_label'] = data['label'].apply(lambda x: '__label__1 ' if x is True else '__label__0 ')

In [176]:
data.head()

,label,text,clean_text,is_ascii,letters,words,stemed_text,new_label
0,1,"To an entire generation of filmgoers, it just might represent the most significant leap in storytelling that they will ever see...",entire generation filmgoers might represent significant leap storytelling ever see,True,82,10,entir gener filmgoer might repres signific leap storytel ever see,__label__0
1,1,Pixar classic is one of the best kids' movies of all time.,pixar classic one best kids movies time,True,39,7,pixar classic one best kid movi time,__label__0
2,1,"Apesar de representar um imenso avanço tecnológico, a força do filme reside no carisma de seus personagens e no charme de sua história.",apesar de representar um imenso avanço tecnológico força filme reside carisma de seus personagens charme de sua história,False,120,18,apesar de representar um imenso avanço tecnológico força film resid carisma de seu personagen charm de sua história,__label__0
3,1,"When Woody perks up in the opening scene, it's not only the toy cowboy who comes alive - we're watching the rebirth of an art form.",woody perks opening scene toy cowboy comes alive watching rebirth art form,True,74,12,woodi perk open scene toy cowboy come aliv watch rebirth art form,__label__0
4,1,Introduced not one but two indelible characters to the pop culture pantheon: cowboy rag-doll Woody (Tom Hanks) and plastic space ranger Buzz Lightyear (Tim Allen). [Blu-ray],introduced one two indelible characters pop culture pantheon cowboy rag doll woody plastic space ranger buzz lightyear,True,118,17,introduc one two indel charact pop cultur pantheon cowboy rag doll woodi plastic space ranger buzz lightyear,__label__0


In [177]:
data[data['is_ascii'] == True].shape

(100314, 8)

In [189]:
data_cleaned = data[data['is_ascii'] == True]
data_cleaned = data_cleaned.ix[:, ['new_label', 'stemed_text']]

In [190]:
data_cleaned.head()

,new_label,stemed_text
0,__label__0,entir gener filmgoer might repres signific leap storytel ever see
1,__label__0,pixar classic one best kid movi time
3,__label__0,woodi perk open scene toy cowboy come aliv watch rebirth art form
4,__label__0,introduc one two indel charact pop cultur pantheon cowboy rag doll woodi plastic space ranger buzz lightyear
5,__label__0,easi see virtual everyth good anim right small seed toy stori


In [188]:
from fasttext import supervised

def learn_ft(dataframe, model_name = 'model', label_prefix='__label__')
    classifier = supervised(file_name, model_name, label_prefix=label_prefix)

(100314, 2)

In [ ]:
result = classifier.test('train.txt')

print('Precision:', result.precision)
print('Recall:', result.recall)
print('Number of examples:', result.nexamples)

In [ ]:
prediction = classifier.predict_proba(list(data_cleaned['stemed_text']))
prediction[:2]

In [ ]:
clf.fit(X_train, y_train)

print("="*30)
print(name)

print('****Results****')
train_predictions = clf.predict(X_test)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))

train_predictions = clf.predict_proba(X_test)
ll = log_loss(y_test, train_predictions)
print("Log Loss: {}".format(ll))

log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
log = log.append(log_entry)

In [ ]:
X = data_cleaned['stemed_text']
y = data_cleaned['new_label']

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

train_index, test_index = list(sss.split(X, y))[0]
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]